In [ ]:
from google.colab import files
uploaded = files.upload()

Saving all_kindle_review.csv to all_kindle_review.csv


In [ ]:
import pandas as pd
import io

df = pd.read_csv(io.BytesIO(uploaded['all_kindle_review.csv']))
print(df)

       Unnamed: 0.1  Unnamed: 0        asin  helpful  rating  \
0                 0       11539  B0033UV8HI  [8, 10]       3   
1                 1        5957  B002HJV4DE   [1, 1]       5   
2                 2        9146  B002ZG96I4   [0, 0]       3   
3                 3        7038  B002QHWOEU   [1, 3]       3   
4                 4        1776  B001A06VJ8   [0, 1]       4   
...             ...         ...         ...      ...     ...   
11995         11995        2183  B001DUGORO   [0, 0]       4   
11996         11996        6272  B002JCSFSQ   [2, 2]       5   
11997         11997       12483  B0035N1V7K   [0, 1]       3   
11998         11998        3640  B001W1XT40   [1, 2]       1   
11999         11999       11398  B003370JUS   [5, 6]       3   

                                              reviewText   reviewTime  \
0      Jace Rankin may be short, but he's nothing to ...   09 2, 2010   
1      Great short read.  I didn't want to put it dow...   10 8, 2013   
2      I'll 

In [ ]:
df = df[['reviewText', 'rating']]
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [ ]:
df.shape

(12000, 2)

In [ ]:
df.isnull().sum()

reviewText    0
rating        0
dtype: int64

In [ ]:
df['rating'].unique()

array([3, 5, 4, 2, 1])

In [ ]:
df['rating'].value_counts()

rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64

In [ ]:
df['rating'] = df['rating'].apply(lambda x : 0 if x < 3 else 1)

In [ ]:
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",1
1,Great short read. I didn't want to put it dow...,1
2,I'll start by saying this is the first of four...,1
3,Aggie is Angela Lansbury who carries pocketboo...,1
4,I did not expect this type of book to be in li...,1


In [ ]:
df['rating'].unique()

array([1, 0])

In [ ]:
df['rating'].value_counts()

rating
1    8000
0    4000
Name: count, dtype: int64

In [ ]:
#lower cases -->
df['reviewText'] = df['reviewText'].str.lower()

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
# Removing special characters
df['reviewText']=df['reviewText'].apply(lambda x:re.sub('[^a-z A-z 0-9-]+', '',x))
# Remove the stopswords
df['reviewText']=df['reviewText'].apply(lambda x:" ".join([y for y in x.split() if y not in stopwords.words('english')]))
# Remove url
df['reviewText']=df['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))
# Remove html tags
df['reviewText']=df['reviewText'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())
# Remove any additional spaces
df['reviewText']=df['reviewText'].apply(lambda x: " ".join(x.split()))

<ipython-input-33-0f0aae6bc507>:8: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  df['reviewText']=df['reviewText'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())


In [ ]:
df.head()

,reviewText,rating
0,jace rankin may short hes nothing mess man hau...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four books wasnt expect...,1
3,aggie angela lansbury carries pocketbooks inst...,1
4,expect type book library pleased find price right,1


In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

In [ ]:
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
df['reviewText']=df['reviewText'].apply(lambda x:lemmatize_words(x))

In [ ]:
df.head()

,reviewText,rating
0,jace rankin may short he nothing mess man haul...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four book wasnt expecti...,1
3,aggie angela lansbury carry pocketbook instead...,1
4,expect type book library pleased find price right,1


# **Train** **and Test the** **model**

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df['reviewText'],df['rating'],test_size=0.20)

**Bag of words and tf-idf**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
bow=CountVectorizer()
X_train_bow=bow.fit_transform(X_train).toarray()
X_test_bow=bow.transform(X_test).toarray()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
X_train_tfidf=tfidf.fit_transform(X_train).toarray()
X_test_tfidf=tfidf.transform(X_test).toarray()

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb_model_bow=GaussianNB().fit(X_train_bow,y_train)
nb_model_tfidf=GaussianNB().fit(X_train_tfidf,y_train)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [ ]:
y_pred_bow=nb_model_bow.predict(X_test_bow)

In [ ]:
y_pred_tfidf=nb_model_bow.predict(X_test_tfidf)

In [ ]:
confusion_matrix(y_test,y_pred_bow)

array([[530, 274],
       [692, 904]])

### **Bag of words accuracy**

In [ ]:
print("BOW accuracy: ",accuracy_score(y_test,y_pred_bow))

BOW accuracy:  0.5975


In [ ]:
confusion_matrix(y_test,y_pred_tfidf)

array([[522, 282],
       [687, 909]])

### **TFIDF Accuracy**

In [ ]:
print("TFIDF accuracy: ",accuracy_score(y_test,y_pred_tfidf))

TFIDF accuracy:  0.59625


# **Word2Vec**

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

X_train_tokens = [word_tokenize(text) for text in X_train]
X_test_tokens = [word_tokenize(text) for text in X_test]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from gensim.models import Word2Vec

# Train the model on your training data
w2v_model = Word2Vec(sentences=X_train_tokens, vector_size=100, window=5, min_count=1, workers=4)

# Save the model to disk (optional)
#w2v_model.save("word2vec.model")

In [ ]:
import numpy as np

def vectorize_text(tokens, model):
    # Get the vectors for words present in the model's vocabulary
    word_vectors = [model.wv[word] for word in tokens if word in model.wv]

    if len(word_vectors) == 0:  # Handle case where no words are in the model's vocabulary
        return np.zeros(model.vector_size)

    # Calculate the mean of the word vectors
    return np.mean(word_vectors, axis=0)

X_train_w2v = np.array([vectorize_text(tokens, w2v_model) for tokens in X_train_tokens])
X_test_w2v = np.array([vectorize_text(tokens, w2v_model) for tokens in X_test_tokens])


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Train the logistic regression model
logreg_model = LogisticRegression(max_iter=1000).fit(X_train_w2v, y_train)

# Predict on the test set
y_pred_logreg = logreg_model.predict(X_test_w2v)

# Evaluate the model
accuracy_logreg = accuracy_score(y_test, y_pred_logreg)
print("Logistic Regression Accuracy:", accuracy_logreg)
print("Logistic Regression Classification Report:")
print(classification_report(y_test, y_pred_logreg))

Logistic Regression Accuracy: 0.7575
Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       0.70      0.48      0.57       804
           1       0.77      0.90      0.83      1596

    accuracy                           0.76      2400
   macro avg       0.74      0.69      0.70      2400
weighted avg       0.75      0.76      0.74      2400



In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# Train the gradient boosting classifier
gbc_model = GradientBoostingClassifier().fit(X_train_w2v, y_train)

# Predict on the test set
y_pred_gbc = gbc_model.predict(X_test_w2v)

# Evaluate the model
accuracy_gbc = accuracy_score(y_test, y_pred_gbc)
print("Gradient Boosting Classifier Accuracy:", accuracy_gbc)
print("Gradient Boosting Classifier Classification Report:")
print(classification_report(y_test, y_pred_gbc))

Gradient Boosting Classifier Accuracy: 0.7525
Gradient Boosting Classifier Classification Report:
              precision    recall  f1-score   support

           0       0.68      0.50      0.57       804
           1       0.78      0.88      0.83      1596

    accuracy                           0.75      2400
   macro avg       0.73      0.69      0.70      2400
weighted avg       0.74      0.75      0.74      2400



# **Best accuracy of model**

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid_logreg = {
    'C': [0.1, 1, 10, 100],
    'solver': ['liblinear', 'lbfgs']
}

grid_search_logreg = GridSearchCV(LogisticRegression(max_iter=1000), param_grid_logreg, cv=5, n_jobs=-1)
grid_search_logreg.fit(X_train_w2v, y_train)

best_logreg_model = grid_search_logreg.best_estimator_

# Predict on the test set
y_pred_best_logreg = best_logreg_model.predict(X_test_w2v)

# Evaluate the model
accuracy_best_logreg = accuracy_score(y_test, y_pred_best_logreg)
print("Best Logistic Regression Accuracy:", accuracy_best_logreg)
print("Best Logistic Regression Classification Report:")
print(classification_report(y_test, y_pred_best_logreg))

Best Logistic Regression Accuracy: 0.78125
Best Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.56      0.63       804
           1       0.80      0.89      0.84      1596

    accuracy                           0.78      2400
   macro avg       0.76      0.73      0.74      2400
weighted avg       0.78      0.78      0.77      2400

